<a href="https://colab.research.google.com/github/SMN-SENAMHI/ADAPTACION/blob/main/CMIP6-VERIFICACION_v1.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# BLOQUE 0 – Instalación y librerías
# ============================================================
!pip install -q intake intake-esm xmip

import intake
import pandas as pd
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.8/306.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.5 

In [ ]:
# ============================================================
# BLOQUE 1 – Abrir catálogo CMIP6
# ============================================================
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
cat = intake.open_esm_datastore(cat_url)

In [ ]:
# ============================================================
# BLOQUE 2 – Parámetros del modelo
# ============================================================
modelo = "CNRM-CM6-1"
experimento = "ssp245"
miembro = "r1i1p1f2"
variables = ["huss","tas","tos","uas","vas",
             "hus","ps","ta","ua","va",
             "psl","ts","mrsol","sftlf","tsl"]

In [ ]:
# ============================================================
# BLOQUE 3 – Búsqueda por variable
# ============================================================
resultados = []

for var in variables:
    cat_filt = cat.search(
        source_id=modelo,
        experiment_id=experimento,
        variable_id=var,
        member_id=miembro
    )

    if len(cat_filt.df) > 0:
        df = cat_filt.df
        tablas = df["table_id"].unique().tolist()
        frecuencias = df["frequency"].unique().tolist() if "frequency" in df else []

        # Intentar capturar rango temporal de diferentes columnas posibles
        inicio = None
        fin = None

        if "time_range" in df.columns:
            start = df["time_range"].apply(lambda x: x.split("-")[0] if isinstance(x, str) else None).dropna()
            end = df["time_range"].apply(lambda x: x.split("-")[-1] if isinstance(x, str) else None).dropna()
            inicio, fin = start.min(), end.max()

        elif "datetime_start" in df.columns and "datetime_end" in df.columns:
            inicio = str(df["datetime_start"].min())[:10]
            fin = str(df["datetime_end"].max())[:10]

        elif "date_range" in df.columns:
            start = df["date_range"].apply(lambda x: x.split("-")[0] if isinstance(x, str) else None).dropna()
            end = df["date_range"].apply(lambda x: x.split("-")[-1] if isinstance(x, str) else None).dropna()
            inicio, fin = start.min(), end.max()

        resultados.append({
            "variable": var,
            "disponible": "✅",
            "table_id": tablas,
            "frecuencia": frecuencias,
            "periodo_inicio": inicio,
            "periodo_fin": fin
        })
    else:
        resultados.append({
            "variable": var,
            "disponible": "❌",
            "table_id": None,
            "frecuencia": None,
            "periodo_inicio": None,
            "periodo_fin": None
        })

df_result = pd.DataFrame(resultados)
display(df_result)

,variable,disponible,table_id,frecuencia,periodo_inicio,periodo_fin
0,huss,✅,"[day, 3hr, Amon]",[],None,None
1,tas,✅,"[day, 3hr, Amon]",[],None,None
2,tos,✅,"[Omon, Oday]",[],None,None
3,uas,✅,"[day, Amon]",[],None,None
4,vas,✅,"[day, Amon]",[],None,None
5,hus,✅,"[day, Amon]",[],None,None
6,ps,✅,[Amon],[],None,None
7,ta,✅,[Amon],[],None,None
8,ua,✅,"[day, Amon]",[],None,None
9,va,✅,"[day, Amon]",[],None,None


In [ ]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
cat = intake.open_esm_datastore(cat_url)
cat.df


,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,ps,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,<NA>,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rsds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,<NA>,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlus,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,<NA>,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,<NA>,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,<NA>,20170706
...,...,...,...,...,...,...,...,...,...,...,...
514813,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,tas,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,<NA>,20211207
514814,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,tauu,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,<NA>,20211207
514815,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,hur,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,<NA>,20211207
514816,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,historical,r1i1p1f1,Amon,hus,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,<NA>,20211207


In [ ]:
import intake
import pandas as pd

# Cargar catálogo
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
cat = intake.open_esm_datastore(cat_url)

# Definir filtros
filtro = (
    (cat.df["activity_id"] == "ScenarioMIP") &
    (cat.df["institution_id"] == "CNRM-CERFACS") &
    (cat.df["source_id"] == "CNRM-CM6-1") &
    (cat.df["experiment_id"] == "ssp245") &
    (cat.df["member_id"] == "r1i1p1f2") &
    (cat.df["variable_id"].isin(["huss","tas","tos","uas","vas",
                                 "hus","ps","ta","ua","va",
                                 "psl","ts","mrsol","sftlf","tsl"]))
)

# Aplicar filtro
df_sub = cat.df.loc[filtro, [
    "activity_id","institution_id","source_id",
    "experiment_id","member_id","table_id",
    "variable_id","grid_label","zstore","version"
]]

# Mostrar resumen
print(f"Total variables encontradas: {len(df_sub)}")
display(df_sub)


Total variables encontradas: 24


,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,version
54522,ScenarioMIP,CNRM-CERFACS,CNRM-CM6-1,ssp245,r1i1p1f2,day,va,gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,20190219
54523,ScenarioMIP,CNRM-CERFACS,CNRM-CM6-1,ssp245,r1i1p1f2,day,uas,gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,20190219
54524,ScenarioMIP,CNRM-CERFACS,CNRM-CM6-1,ssp245,r1i1p1f2,day,ua,gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,20190219
54527,ScenarioMIP,CNRM-CERFACS,CNRM-CM6-1,ssp245,r1i1p1f2,day,tas,gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,20190219
54533,ScenarioMIP,CNRM-CERFACS,CNRM-CM6-1,ssp245,r1i1p1f2,day,psl,gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,20190219
54538,ScenarioMIP,CNRM-CERFACS,CNRM-CM6-1,ssp245,r1i1p1f2,day,huss,gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,20190219
54539,ScenarioMIP,CNRM-CERFACS,CNRM-CM6-1,ssp245,r1i1p1f2,day,hus,gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,20190219
54541,ScenarioMIP,CNRM-CERFACS,CNRM-CM6-1,ssp245,r1i1p1f2,day,vas,gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,20190219
54622,ScenarioMIP,CNRM-CERFACS,CNRM-CM6-1,ssp245,r1i1p1f2,3hr,tas,gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,20190219
54626,ScenarioMIP,CNRM-CERFACS,CNRM-CM6-1,ssp245,r1i1p1f2,3hr,huss,gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,20190219


In [ ]:
import intake
import pandas as pd

# ============================================================
# BLOQUE 0 – Abrir catálogo CMIP6 (público)
# ============================================================
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
cat = intake.open_esm_datastore(cat_url)

# ============================================================
# BLOQUE 1 – Filtro del modelo y experimento
# ============================================================
modelo = "CNRM-CM6-1"
institucion = "CNRM-CERFACS"
experimento = "ssp585"
miembro = "r1i1p1f2"

filtro = (
    (cat.df["activity_id"] == "ScenarioMIP") &
    (cat.df["institution_id"] == institucion) &
    (cat.df["source_id"] == modelo) &
    (cat.df["experiment_id"] == experimento) &
    (cat.df["member_id"] == miembro)
)

# Extraer variables reales disponibles en el catálogo
df_vars = cat.df.loc[filtro, ["variable_id","table_id","grid_label","version"]].drop_duplicates().sort_values("variable_id")

print(f"Variables encontradas: {len(df_vars)}\n")

# Mostrar listado simple tipo "variable → tabla"
for v, t in zip(df_vars["variable_id"], df_vars["table_id"]):
    print(f"{v:10s} → {t}")




Variables encontradas: 158

areacello  → Ofx
bigthetao  → Omon
bigthetaoga → Omon
cl         → Amon
cli        → Amon
clivi      → Amon
clt        → Amon
clt        → day
clw        → Amon
clwvi      → Amon
evspsbl    → Amon
evspsblpot → Emon
evspsblveg → Lmon
ficeberg   → Omon
gpp        → Lmon
hfbasin    → Omon
hfds       → Omon
hfls       → Amon
hfls       → day
hfss       → Amon
hfss       → day
hur        → Amon
hurs       → Amon
hurs       → day
hursmax    → day
hursmin    → day
hus        → 6hrLev
hus        → Amon
hus        → day
huss       → 3hr
huss       → Amon
huss       → day
lai        → Lmon
masscello  → Omon
masso      → Omon
mfo        → Omon
mlotst     → Omon
mrro       → Lmon
mrros      → Lmon
mrso       → day
mrso       → Lmon
mrsos      → Lmon
npp        → Lmon
o3         → Amon
od550aer   → AERmon
pbo        → Omon
pr         → day
pr         → 3hr
pr         → Amon
prc        → day
prc        → Amon
prsn       → day
prsn       → Amon
prveg      → Lmon
prw       

In [ ]:
import intake
import pandas as pd

# ============================================================
# BLOQUE 0 – Cargar catálogo CMIP6
# ============================================================
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
cat = intake.open_esm_datastore(cat_url)

# ============================================================
# BLOQUE 1 – Parámetros personalizables
# ============================================================
# 🔹 Editar según el modelo / escenario que se desea analizar
institucion = "CNRM-CERFACS"     # Ejemplo: "MOHC", "GFDL", "ECCC"
modelo = "CNRM-CM6-1"            # Ejemplo: "UKESM1-0-LL", "GFDL-ESM4"
experimento = "ssp245"           # Ejemplo: "historical", "ssp585"
miembro = "r1i1p1f2"             # Ejemplo: "r1i1p1f1", "r2i1p1f1"

# ============================================================
# BLOQUE 2 – Filtrar catálogo según parámetros
# ============================================================
filtro = (
    (cat.df["institution_id"] == institucion) &
    (cat.df["source_id"] == modelo) &
    (cat.df["experiment_id"] == experimento) &
    (cat.df["member_id"] == miembro)
)

df = cat.df.loc[filtro, ["variable_id", "table_id", "grid_label", "zstore"]].drop_duplicates()

if df.empty:
    raise ValueError(f"No se encontraron datos para {institucion} / {modelo} / {experimento} / {miembro}")

# ============================================================
# BLOQUE 3 – Variables de interés para WRF
# ============================================================
uso_WRF = {
    "ua": "Campo 3D – Viento U",
    "va": "Campo 3D – Viento V",
    "ta": "Campo 3D – Temperatura",
    "hus": "Campo 3D – Humedad específica",
    "hur": "Campo 3D – Humedad relativa",
    "zg": "Campo 3D – Geopotencial",
    "ps": "Presión de superficie",
    "psl": "Presión al nivel del mar (diagnóstico)",
    "tas": "Temperatura 2 m (validación/nudging)",
    "huss": "Humedad 2 m (validación/nudging)",
    "uas": "Viento 10 m U (superficie)",
    "vas": "Viento 10 m V (superficie)",
    "tos": "SST – Temperatura superficial del mar",
    "ts": "Temperatura de superficie terrestre",
    "tsl": "Temperatura de suelo",
    "mrso": "Humedad total del suelo",
    "mrsos": "Humedad del suelo (superficie)",
    "sftlf": "Máscara tierra-océano"
}

# ============================================================
# BLOQUE 4 – Definir frecuencias de interés
# ============================================================
frecs_sub = [ "6hrPlevPt", "CF3hr", "3hrPt","6hrPt","3hr", "6hr", "6hrLev",  "subhr"]
frecs_mon = ["mon", "amon", "omon", "lmon", "limon", "simon", "aermon", "emon"]

# ============================================================
# BLOQUE 5 – Comprobar disponibilidad y frecuencia detectada
# ============================================================
resultados = []
for var, uso in uso_WRF.items():
    subset = df.loc[df["variable_id"] == var]
    tablas = subset["table_id"].tolist()
    grids = subset["grid_label"].tolist()
    urls = subset["zstore"].tolist()

    sub = any(any(f in t.lower() for f in frecs_sub) for t in tablas)
    mon = any(any(f in t.lower() for f in frecs_mon) for t in tablas)
    freq_detectada = next((t for t in tablas if any(f in t.lower() for f in frecs_sub)), "(no subdiaria)")
    grid_ejemplo = grids[0] if len(grids) > 0 else "(sin grilla)"
    url_ejemplo = urls[0] if len(urls) > 0 else "(sin URL)"

    resultados.append({
        "variable_id": var,
        "uso_WRF": uso,
        "tablas_encontradas": ", ".join(tablas) if tablas else "(no disponible)",
        "frecuencia_detectada": freq_detectada,
        "grid_label": grid_ejemplo,
        "zstore": url_ejemplo,
        "subdiaria": "✅ Sí" if sub else "❌ No",
        "mensual": "✅ Sí" if mon else "❌ No"
    })

df_out = pd.DataFrame(resultados)

# ============================================================
# BLOQUE 6 – Clasificar por tipo físico
# ============================================================
campos_3d = ["ua", "va", "ta", "hus", "hur", "zg"]
campos_suelo = ["tsl", "mrso", "mrsos", "sftlf", "ts"]

def clasificar_tipo(var):
    if var in campos_3d:
        return "niveles"
    elif var in campos_suelo:
        return "suelo"
    else:
        return "superficie"

df_out["tipo_variable"] = df_out["variable_id"].apply(clasificar_tipo)
orden = {"niveles": 0, "superficie": 1, "suelo": 2}
df_out = df_out.sort_values(by=["tipo_variable", "variable_id"], key=lambda col: col.map(orden))

# ============================================================
# BLOQUE 7 – Marcar “WRF_Listo” (solo subdiarias)
# ============================================================
df_out["WRF_Listo"] = df_out["subdiaria"].apply(lambda x: "✅ Sí" if "Sí" in x else "❌ No")

# ============================================================
# BLOQUE 8 – Mostrar tabla final
# ============================================================
cols = [
    "variable_id",
    "uso_WRF",
    "tablas_encontradas",
    "frecuencia_detectada",
    "grid_label",
    "zstore",
    "subdiaria",
    "mensual",
    "WRF_Listo"
]

display(df_out[cols])






,variable_id,uso_WRF,tablas_encontradas,frecuencia_detectada,grid_label,zstore,subdiaria,mensual,WRF_Listo
0,ua,Campo 3D – Viento U,"day, Amon",(no subdiaria),gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,❌ No,✅ Sí,❌ No
1,va,Campo 3D – Viento V,"day, Amon",(no subdiaria),gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,❌ No,✅ Sí,❌ No
2,ta,Campo 3D – Temperatura,Amon,(no subdiaria),gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,❌ No,✅ Sí,❌ No
3,hus,Campo 3D – Humedad específica,"day, Amon",(no subdiaria),gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,❌ No,✅ Sí,❌ No
4,hur,Campo 3D – Humedad relativa,Amon,(no subdiaria),gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,❌ No,✅ Sí,❌ No
5,zg,Campo 3D – Geopotencial,"day, Amon",(no subdiaria),gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,❌ No,✅ Sí,❌ No
6,ps,Presión de superficie,Amon,(no subdiaria),gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,❌ No,✅ Sí,❌ No
7,psl,Presión al nivel del mar (diagnóstico),"day, Amon",(no subdiaria),gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,❌ No,✅ Sí,❌ No
8,tas,Temperatura 2 m (validación/nudging),"day, 3hr, Amon",3hr,gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,✅ Sí,✅ Sí,✅ Sí
9,huss,Humedad 2 m (validación/nudging),"day, 3hr, Amon",3hr,gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,✅ Sí,✅ Sí,✅ Sí


In [ ]:
# ============================================================
# VERIFICACIÓN DE VARIABLES CMIP6 PARA WRF
# Compatible con CMIP, ScenarioMIP, AMIP, DAMIP, etc.
# ============================================================

import intake
import pandas as pd

# ============================================================
# BLOQUE 0 – Conectar al catálogo PANGEO-CMIP6
# ============================================================
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
cat = intake.open_esm_datastore(cat_url)

# ============================================================
# BLOQUE 1 – Parámetros editables del usuario
# ============================================================
activity = "ScenarioMIP"          # Ej: "CMIP", "ScenarioMIP", "DAMIP"
institucion = "CNRM-CERFACS"      # Ej: "MOHC", "GFDL", "MPI-M"
modelo = "CNRM-CM6-1-HR"          # Ej: "UKESM1-0-LL", "GFDL-ESM4"
experimento = "ssp585"            # Ej: "historical", "ssp245", "ssp585"
miembro = "r1i1p1f2"              # Ej: "r1i1p1f1", "r2i1p1f1"

# ============================================================
# BLOQUE 2 – Filtrar catálogo según parámetros
# ============================================================
filtro = (
    (cat.df["activity_id"] == activity) &
    (cat.df["institution_id"] == institucion) &
    (cat.df["source_id"] == modelo) &
    (cat.df["experiment_id"] == experimento) &
    (cat.df["member_id"] == miembro)
)

df = cat.df.loc[filtro, ["variable_id", "table_id", "grid_label", "zstore"]].drop_duplicates()

if df.empty:
    raise ValueError(f"No se encontraron datos para {activity}/{institucion}/{modelo}/{experimento}/{miembro}")

# ============================================================
# BLOQUE 3 – Variables requeridas para forzar WRF
# ============================================================
uso_WRF = {
    "ua": "Campo 3D – Viento U",
    "va": "Campo 3D – Viento V",
    "ta": "Campo 3D – Temperatura",
    "hus": "Campo 3D – Humedad específica",
    "hur": "Campo 3D – Humedad relativa",
    "zg": "Campo 3D – Geopotencial",
    "ps": "Presión de superficie",
    "psl": "Presión al nivel del mar (diagnóstico)",
    "tas": "Temperatura 2 m",
    "huss": "Humedad 2 m",
    "uas": "Viento 10 m U",
    "vas": "Viento 10 m V",
    "tos": "Temperatura superficial del mar (SST)",
    "ts": "Temperatura de superficie terrestre",
    "tsl": "Temperatura de suelo",
    "mrso": "Humedad total del suelo",
    "mrsos": "Humedad del suelo (superficie)",
    "sftlf": "Máscara tierra-océano"
}

# ============================================================
# BLOQUE 4 – Definir frecuencias
# ============================================================
frecs_sub = ["3hr", "6hr", "6hrLev", "6hrPlevPt", "subhr", "6hrPt"]
frecs_mon = ["mon", "amon", "omon", "lmon", "limon", "simon", "aermon", "emon"]

# ============================================================
# BLOQUE 5 – Verificación por variable
# ============================================================
resultados = []
for var, uso in uso_WRF.items():
    subset = df.loc[df["variable_id"] == var]
    tablas = subset["table_id"].tolist()
    grids = subset["grid_label"].tolist()
    urls = subset["zstore"].tolist()

    sub = any(any(f in t.lower() for f in frecs_sub) for t in tablas)
    mon = any(any(f in t.lower() for f in frecs_mon) for t in tablas)
    freq_detectada = next((t for t in tablas if any(f in t.lower() for f in frecs_sub)), "(no subdiaria)")
    grid_ejemplo = grids[0] if len(grids) > 0 else "(sin grilla)"
    url_ejemplo = urls[0] if len(urls) > 0 else "(sin URL)"

    resultados.append({
        "variable_id": var,
        "uso_WRF": uso,
        "tablas_encontradas": ", ".join(tablas) if tablas else "(no disponible)",
        "frecuencia_detectada": freq_detectada,
        "grid_label": grid_ejemplo,
        "zstore": url_ejemplo,
        "subdiaria": "✅ Sí" if sub else "❌ No",
        "mensual": "✅ Sí" if mon else "❌ No"
    })

df_out = pd.DataFrame(resultados)

# ============================================================
# BLOQUE 6 – Clasificación física
# ============================================================
campos_3d = ["ua", "va", "ta", "hus", "hur", "zg"]
campos_suelo = ["tsl", "mrso", "mrsos", "sftlf", "ts"]

def clasificar_tipo(var):
    if var in campos_3d:
        return "niveles"
    elif var in campos_suelo:
        return "suelo"
    else:
        return "superficie"

df_out["tipo_variable"] = df_out["variable_id"].apply(clasificar_tipo)
orden = {"niveles": 0, "superficie": 1, "suelo": 2}
df_out = df_out.sort_values(by=["tipo_variable", "variable_id"], key=lambda col: col.map(orden))

# ============================================================
# BLOQUE 7 – Marcar si está lista para WRF
# ============================================================
df_out["WRF_Listo"] = df_out["subdiaria"].apply(lambda x: "✅ Sí" if "Sí" in x else "❌ No")

# ============================================================
# BLOQUE 8 – Mostrar tabla final
# ============================================================
cols = [
    "variable_id",
    "uso_WRF",
    "tablas_encontradas",
    "frecuencia_detectada",
    "grid_label",
    "zstore",
    "subdiaria",
    "mensual",
    "WRF_Listo"
]

display(df_out[cols])




,variable_id,uso_WRF,tablas_encontradas,frecuencia_detectada,grid_label,zstore,subdiaria,mensual,WRF_Listo
0,ua,Campo 3D – Viento U,"Amon, day",(no subdiaria),gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,❌ No,✅ Sí,❌ No
1,va,Campo 3D – Viento V,"Amon, day",(no subdiaria),gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,❌ No,✅ Sí,❌ No
2,ta,Campo 3D – Temperatura,Amon,(no subdiaria),gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,❌ No,✅ Sí,❌ No
3,hus,Campo 3D – Humedad específica,"day, Amon",(no subdiaria),gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,❌ No,✅ Sí,❌ No
4,hur,Campo 3D – Humedad relativa,"day, Amon",(no subdiaria),gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,❌ No,✅ Sí,❌ No
5,zg,Campo 3D – Geopotencial,"Amon, day",(no subdiaria),gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,❌ No,✅ Sí,❌ No
6,ps,Presión de superficie,Amon,(no subdiaria),gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,❌ No,✅ Sí,❌ No
7,psl,Presión al nivel del mar (diagnóstico),"Amon, day",(no subdiaria),gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,❌ No,✅ Sí,❌ No
8,tas,Temperatura 2 m,"Amon, 3hr, day",3hr,gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,✅ Sí,✅ Sí,✅ Sí
9,huss,Humedad 2 m,"Amon, 3hr, day",3hr,gr,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,✅ Sí,✅ Sí,✅ Sí


In [ ]:
query = (
    "https://esg1.umr-cnrm.fr/esg-search/search/"
    "?project=CMIP6&source_id=CNRM-CM6-1-HR&experiment_id=ssp585"
    "&variable_id=psl&member_id=r1i1p1f2&table_id=6hrPlevPt"
    "&distrib=true&format=application%2Fjson"
)


In [ ]:
for table in ["6hrPt", "6hrPlevPt"]:
    print(table, check_esgf("psl", "CNRM-CM6-1-HR", "ssp585", "r1i1p1f2", table))


6hrPt False
6hrPlevPt False
